In [1]:
!pip install -q transformers datasets peft accelerate evaluate peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 36.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# =========================
# 0) SETUP & INSTALLS
# =========================

import os, math, json, random, functools, collections
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from peft import AdaLoraConfig, get_peft_model, TaskType
from transformers import DataCollatorWithPadding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 7
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

2025-11-17 09:29:42.568758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763371782.734490      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763371782.783358      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# =========================
# SICK-E from Kaggle files
# =========================
data_files = {
    "train": "/kaggle/input/sick-e/SICK_train.txt",
    "validation": "/kaggle/input/sick-e/SICK_trial.txt",  # use trial as dev
    "test": "/kaggle/input/sick-e/SICK_test.txt",
}

# The files are tab-separated with a header row
raw = load_dataset("csv", data_files=data_files, delimiter="\t")

# 3-way NLI labels
label2id = {"ENTAILMENT": 0, "NEUTRAL": 1, "CONTRADICTION": 2}
id2label = {v: k for k, v in label2id.items()}

def preprocess(example):
    # example keys: "sentence_A", "sentence_B", "entailment_judgment", ...
    enc = tokenizer(
        example["sentence_A"],
        example["sentence_B"],
        truncation=True,
        max_length=256,
    )
    # handle batched=True
    labels = example["entailment_judgment"]
    if isinstance(labels, list):
        enc["label"] = [label2id[l] for l in labels]
    else:
        enc["label"] = label2id[labels]
    return enc

train_ds = raw["train"].map(preprocess, batched=True)
valid_ds = raw["validation"].map(preprocess, batched=True)
test_ds  = raw["test"].map(preprocess, batched=True)

cols = ["input_ids", "attention_mask", "label"]
train_ds.set_format(type="torch", columns=cols)
valid_ds.set_format(type="torch", columns=cols)
test_ds.set_format(type="torch", columns=cols)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4927 [00:00<?, ? examples/s]

In [4]:
# =========================
# 2) TARGET MODULES (All attention + FFN)
#    Wq, Wk, Wv, Wo, Wf1, Wf2
# =========================
# We'll rely on module name patterns in RoBERTa:
#  roberta.encoder.layer.{i}.attention.self.{query,key,value}
#  roberta.encoder.layer.{i}.attention.output.dense       -> Wo
#  roberta.encoder.layer.{i}.intermediate.dense           -> Wf1
#  roberta.encoder.layer.{i}.output.dense (NOT attention) -> Wf2

TARGET_PATTERNS = ["attention.self.query",
                   "attention.self.key",
                   "attention.self.value",
                   "attention.output.dense",  # Wo
                   "intermediate.dense",      # Wf1
                   "output.dense"]            # Wf2 (feedforward output)

# Helper to test if a module full name is one we want (and classify its shape-group)
def classify_module_name(full_name):
    # returns (group_key, pretty_tag) or (None, None)
    # group_key used for sharing; pretty_tag displayed to user
    if ".attention.self.query" in full_name:
        return ("Wq", "Wq")
    if ".attention.self.key" in full_name:
        return ("Wk", "Wk")
    if ".attention.self.value" in full_name:
        return ("Wv", "Wv")
    if ".attention.output.dense" in full_name:
        return ("Wo", "Wo")
    # Now, distinguish FFN output.dense vs attention.output.dense
    if ".intermediate.dense" in full_name:
        return ("Wf1", "Wf1")
    if ".output.dense" in full_name and ".attention." not in full_name:
        return ("Wf2", "Wf2")
    return (None, None)


In [5]:
import math

# =========================
# SCHEDULE CONSTANTS
# =========================
# Stage-1 (importance) hyperparams
ALPHA_STAGE1      = 0.10    # 10% subset of train
EPOCHS_STAGE1     = 3
BATCH_SIZE_STAGE1 = 64

# Stage-2 (full training) hyperparams
EPOCHS_STAGE2     = 5
BATCH_SIZE_STAGE2 = 64

N_train = len(train_ds)

# ---------- Stage-1 schedule ----------
K_STAGE1 = max(1, int(ALPHA_STAGE1 * N_train))
STEPS_PER_EPOCH_STAGE1 = math.ceil(K_STAGE1 / BATCH_SIZE_STAGE1)
TOTAL_STEP_STAGE1      = EPOCHS_STAGE1 * STEPS_PER_EPOCH_STAGE1

TINIT_STAGE1  = int(round(0.15 * TOTAL_STEP_STAGE1))
TMID_STAGE1   = int(round(0.40 * TOTAL_STEP_STAGE1))
TFINAL_STAGE1 = TOTAL_STEP_STAGE1 - TINIT_STAGE1 - TMID_STAGE1

print("[Stage-1] N_train:", N_train)
print("[Stage-1] subset K:", K_STAGE1)
print("[Stage-1] steps/epoch:", STEPS_PER_EPOCH_STAGE1)
print("[Stage-1] total_step:", TOTAL_STEP_STAGE1)
print("[Stage-1] tinit:", TINIT_STAGE1, "tfinal:", TFINAL_STAGE1, "middle:", TMID_STAGE1)

# ---------- Stage-2 schedule ----------
STEPS_PER_EPOCH_STAGE2 = math.ceil(N_train / BATCH_SIZE_STAGE2)
TOTAL_STEP_STAGE2      = EPOCHS_STAGE2 * STEPS_PER_EPOCH_STAGE2

TINIT_STAGE2  = int(round(0.15 * TOTAL_STEP_STAGE2))
TMID_STAGE2   = int(round(0.40 * TOTAL_STEP_STAGE2))
TFINAL_STAGE2 = TOTAL_STEP_STAGE2 - TINIT_STAGE2 - TMID_STAGE2

print("[Stage-2] steps/epoch:", STEPS_PER_EPOCH_STAGE2)
print("[Stage-2] total_step:", TOTAL_STEP_STAGE2)
print("[Stage-2] tinit:", TINIT_STAGE2, "tfinal:", TFINAL_STAGE2, "middle:", TMID_STAGE2)


[Stage-1] N_train: 4500
[Stage-1] subset K: 450
[Stage-1] steps/epoch: 8
[Stage-1] total_step: 24
[Stage-1] tinit: 4 tfinal: 10 middle: 10
[Stage-2] steps/epoch: 71
[Stage-2] total_step: 355
[Stage-2] tinit: 53 tfinal: 160 middle: 142


In [6]:
# =========================
# ADALORA CONFIGS
# =========================
ADALORA_STAGE1_CFG = dict(
    target_r = 4,
    init_r   = 6,
    lora_alpha = 16,
    lora_dropout = 0.0,
    beta1 = 0.85, beta2 = 0.85,
    tinit = TINIT_STAGE1,
    tfinal = TFINAL_STAGE1,
    deltaT = 1,
    orth_reg_weight = 0.3,
    total_step = TOTAL_STEP_STAGE1,
)

ADALORA_STAGE2_CFG = dict(
    target_r = 4,
    init_r   = 6,
    lora_alpha = 16,
    lora_dropout = 0.0,
    beta1 = 0.85, beta2 = 0.85,
    tinit = TINIT_STAGE2,
    tfinal = TFINAL_STAGE2,
    deltaT = 1,
    orth_reg_weight = 0.3,
    total_step = TOTAL_STEP_STAGE2,
)


In [7]:
# =========================
# 4) STAGE-1 MODEL: build w/ AdaLoRA on all target modules
# =========================
def build_stage1_model():
    base = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
    base.to(device)
    # Use broad patterns (we filter later when we compute sharing)
    peft_cfg = AdaLoraConfig(
        task_type=TaskType.SEQ_CLS,
        target_modules=["query","key","value","dense"],  # these substrings cover all six
        **ADALORA_STAGE1_CFG
    )
    peft_model = get_peft_model(base, peft_cfg)
    peft_model.print_trainable_parameters()
    return peft_model


In [8]:
# =========================
# 5) STAGE-1 TRAINING on α=0.1 subset + IMPORTANCE via ||ΔW x||^2
#    Implement LoRA-drop importance with AdaLoRA adapters
# =========================
# We'll register forward-pre hooks on every AdaLoRA-wrapped Linear and compute
#  delta = scaling * B(A(x))  (and include AdaLoRA magnitude vector if present)
# Accumulate per-module energy: g_i += ||delta||^2, then normalize to importances I_i
# See LoRA-drop Sec. 3.1/3.2 for this criterion. (We use AdaLoRA to generate Δ.) 


# --- helper: detect AdaLoRA-wrapped linear-like modules ---
def is_adalora_linear(m):
    """
    PEFT's AdaLoRA wraps target layers in SVDLinear, which subclasses
    nn.Module + AdaLoraLayer (not nn.Linear). We detect them by the
    presence of lora_A / lora_B ParameterDicts.
    """
    return hasattr(m, "lora_A") and hasattr(m, "lora_B")

def make_energy_meter(model):
    """
    Register forward-pre hooks on every AdaLoRA/LoRA layer and accumulate
    an importance score per module:
        g_i += || ΔW_i x ||^2  over the warm-up mini-batches.

    For AdaLoRA: ΔW = B diag(E) A
    For LoRA:    ΔW = B A              (no E)
    """
    energy = collections.defaultdict(float)
    hooks = []

    def register_one(full_name, module):
        # classify into Wq/Wk/Wv/Wo/Wf1/Wf2 groups for reporting
        group, tag = classify_module_name(full_name)
        key = (full_name, group or "misc", tag or "misc")

        def pre_hook(mod, inputs):
            x = inputs[0]        # shape: [batch, seq, hidden] or [*, hidden]
            # flatten to 2D: [N, in_features] so F.linear works
            x2 = x.view(-1, x.size(-1))

            # 1) pick an adapter name robustly
            if not hasattr(mod, "lora_A") or len(mod.lora_A) == 0:
                return

            active = getattr(mod, "active_adapter", None)
            if isinstance(active, str) and active in mod.lora_A:
                adapter_name = active
            else:
                adapter_name = next(iter(mod.lora_A.keys()))

            if adapter_name not in mod.lora_B:
                return  # adapter not fully attached

            # 2) Get A and B; they can be nn.Linear or Tensors/Parameters
            A_obj = mod.lora_A[adapter_name]
            B_obj = mod.lora_B[adapter_name]

            # unwrap .weight if these are Linear modules (LoRA case)
            A = A_obj.weight if isinstance(A_obj, nn.Linear) else A_obj
            B = B_obj.weight if isinstance(B_obj, nn.Linear) else B_obj

            # low-rank projection: x2 -> rank space
            # F.linear(x, weight) = x @ weight.T, so weight=[r, in_dim]
            h = F.linear(x2, A)   # [N, r]

            # 3) AdaLoRA-specific singular values E (absent for plain LoRA)
            if hasattr(mod, "lora_E") and adapter_name in mod.lora_E:
                E = mod.lora_E[adapter_name].view(-1)   # [r]
                h = h * E                               # broadcast over batch

            # 4) project back to output space with B: [out_dim, r]
            delta = F.linear(h, B)  # [N, out_dim]

            # 5) LoRA / AdaLoRA scaling (α / r etc.), stored in mod.scaling
            if hasattr(mod, "scaling") and adapter_name in mod.scaling:
                delta = delta * mod.scaling[adapter_name]

            # accumulate energy ||ΔW x||^2 for this module
            e = delta.detach().pow(2).sum().item()
            energy[key] += e

        hooks.append(module.register_forward_pre_hook(pre_hook, with_kwargs=False))

    # attach hooks to all modules that have LoRA/AdaLoRA layers
    for full_name, module in model.named_modules():
        if is_adalora_linear(module):   # this still just checks for lora_A/lora_B
            register_one(full_name, module)

    return energy, hooks



def remove_hooks(hooks):
    for h in hooks: h.remove()

def train_for_importance(
    peft_model,
    alpha=ALPHA_STAGE1,
    epochs=EPOCHS_STAGE1,
    lr=1.2e-3,
    batch_size=BATCH_SIZE_STAGE1,
):
    # sub-sample α of train set
    n = len(train_ds)
    k = max(1, int(alpha * n))
    idx = list(range(n))
    random.shuffle(idx); idx = idx[:k]
    small = train_ds.select(idx)

    dl = DataLoader(
        small,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator,
        drop_last=False,
    )

    # Only AdaLoRA parameters train (base already frozen by PEFT)
    unique_params = {}
    for n, p in peft_model.named_parameters():
        if p.requires_grad:
            unique_params[id(p)] = p
    optim = torch.optim.AdamW(list(unique_params.values()), lr=lr)

    total_steps = epochs * math.ceil(len(dl))
    sched = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps,
    )

    energy, hooks = make_energy_meter(peft_model)
    peft_model.train()

    global_step = 0

    for ep in range(epochs):
        for batch in dl:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = peft_model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["labels"],
            )
            loss = out.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(peft_model.parameters(), 1.0)
            optim.step()

            # AdaLoRA rank allocation in Stage-1
            global_step += 1
            peft_model.base_model.update_and_allocate(global_step)

            sched.step()
            optim.zero_grad()

    remove_hooks(hooks)

    # Normalize importances
    total_e = sum(energy.values()) + 1e-12
    importance = []
    for (full_name, group, tag), val in energy.items():
        importance.append({
            "name": full_name, "group": group, "tag": tag,
            "energy": val, "importance": val / total_e,
        })
    importance.sort(key=lambda d: d["importance"], reverse=True)
    return importance


# Run Stage-1 (warm-up + importance)
stage1_model = build_stage1_model()
importance = train_for_importance(stage1_model, alpha=0.10, epochs=5, lr=1.2e-3, batch_size=64)

def split_importance(importance, threshold):
    cum, important_ids, low_ids = 0.0, [], []
    for item in importance:
        if cum < threshold:
            important_ids.append(item["name"])
            cum += item["importance"]
        else:
            low_ids.append(item["name"])
    return important_ids, low_ids, cum

# Use the same importance list for multiple thresholds
important95_ids, low95_ids, cum95 = split_importance(importance, 0.95)
important90_ids, low90_ids, cum90 = split_importance(importance, 0.90)

print(f"[Stage-1@0.95] selected {len(important95_ids)} important, {len(low95_ids)} low; cum={cum95:.3f}")
print(f"[Stage-1@0.90] selected {len(important90_ids)} important, {len(low90_ids)} low; cum={cum90:.3f}")


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,588,659 || all params: 126,236,670 || trainable%: 1.2585
[Stage-1@0.95] selected 32 important, 40 low; cum=0.952
[Stage-1@0.90] selected 26 important, 46 low; cum=0.907


In [9]:
# Run Stage-1 (warm-up + importance)
stage1_model = build_stage1_model()
importance = train_for_importance(stage1_model, alpha=0.10, epochs=5, lr=1.2e-3, batch_size=64)

def split_importance(importance, threshold):
    cum, important_ids, low_ids = 0.0, [], []
    for item in importance:
        if cum < threshold:
            important_ids.append(item["name"])
            cum += item["importance"]
        else:
            low_ids.append(item["name"])
    return important_ids, low_ids, cum

# Use the same importance list for multiple thresholds
important95_ids, low95_ids, cum95 = split_importance(importance, 0.95)
important90_ids, low90_ids, cum90 = split_importance(importance, 0.90)

print(f"[Stage-1@0.95] selected {len(important95_ids)} important, {len(low95_ids)} low; cum={cum95:.3f}")
print(f"[Stage-1@0.90] selected {len(important90_ids)} important, {len(low90_ids)} low; cum={cum90:.3f}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,588,659 || all params: 126,236,670 || trainable%: 1.2585
[Stage-1@0.95] selected 29 important, 43 low; cum=0.952
[Stage-1@0.90] selected 23 important, 49 low; cum=0.905


In [10]:
# =========================
# 6) STAGE-2 MODEL: rebuild, then tie low-importance modules
#     to one shared AdaLoRA PER SHAPE GROUP
# =========================
def build_stage2_model():
    base = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=3
    ).to(device)
    peft_cfg = AdaLoraConfig(
        task_type=TaskType.SEQ_CLS,
        target_modules=["query", "key", "value", "dense"],  # cover all six types
        **ADALORA_STAGE2_CFG
    )
    model = get_peft_model(base, peft_cfg)
    return model

stage2_model = build_stage2_model()

# Create a shared-parameter bank per group (Wq, Wk, Wv, Wo, Wf1, Wf2)
class SharedAdaLoraBank(nn.Module):
    """
    For each group (Wq/Wk/Wv/Wo/Wf1/Wf2), create a single AdaLoRA
    parameter triple (A, B, E) and tie all LOW-IMPORTANCE modules
    in that group to share them.
    """
    def __init__(self, model, adapter_name="default"):
        super().__init__()
        self.adapter_name = adapter_name

        # Discover one exemplar per group to read shapes from
        exemplars = {}
        for name, m in model.named_modules():
            group, tag = classify_module_name(name)
            if group and group not in exemplars and is_adalora_linear(m) and len(m.lora_A) > 0:
                exemplars[group] = m

        # Build shared A/B/E parameters for each group
        for group, m in exemplars.items():
            # robustly pick adapter name
            adapter = self.adapter_name
            if adapter not in m.lora_A:
                adapter = next(iter(m.lora_A.keys()))

            A = m.lora_A[adapter]  # shape [r, in_dim]
            B = m.lora_B[adapter]  # shape [out_dim, r]
            r, in_dim = A.shape
            out_dim, r2 = B.shape
            assert r == r2

            # register shared A and B
            A_param = nn.Parameter(torch.empty_like(A))
            B_param = nn.Parameter(torch.empty_like(B))
            # init: A ~ Kaiming, B ~ 0 (standard LoRA-style)
            nn.init.kaiming_uniform_(A_param, a=math.sqrt(5))
            nn.init.zeros_(B_param)

            self.register_parameter(f"{group}_A", A_param)
            self.register_parameter(f"{group}_B", B_param)

            # AdaLoRA-specific singular values E (if present)
            if hasattr(m, "lora_E") and adapter in m.lora_E:
                E = m.lora_E[adapter]
                E_param = nn.Parameter(torch.zeros_like(E))  # start at 0
            else:
                E_param = nn.Parameter(torch.zeros(r, 1))
            self.register_parameter(f"{group}_E", E_param)

    def tie_into(self, module, group):
        """
        Replace this module's local A/B/E Parameters with the shared
        ones for its group.
        """
        adapter = self.adapter_name
        if adapter not in module.lora_A:
            adapter = next(iter(module.lora_A.keys()))

        # share A, B, and E
        module.lora_A[adapter] = getattr(self, f"{group}_A")
        module.lora_B[adapter] = getattr(self, f"{group}_B")
        if hasattr(module, "lora_E"):
            module.lora_E[adapter] = getattr(self, f"{group}_E")


# instantiate bank and attach to the PEFT model so its params are counted
shared_bank = SharedAdaLoraBank(stage2_model).to(device)
stage2_model.shared_bank = shared_bank  # important: registers as submodule


# Apply tying for LOW-IMPORTANCE modules: share A/B/E within each group
def apply_sharing_with_bank(model, low_module_names, bank):
    """
    Tie all LOW-IMPORTANCE modules in low_module_names to the given bank.
    """
    name2module = {n: m for n, m in model.named_modules()}
    applied = 0
    for n in low_module_names:
        if n in name2module and is_adalora_linear(name2module[n]):
            group, tag = classify_module_name(n)
            if group is None:
                continue
            bank.tie_into(name2module[n], group)
            applied += 1
    print(f"[Stage-2] Shared parameters tied across {applied} low-importance modules.")



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# =========================
# 8) STAGE-2 TRAIN/VALID/TEST
# =========================
def compute_adalora_budget(model, adapter_name="default", thresh=1e-6):
    """
    Compute theoretical AdaLoRA parameter budget:
      - full_params: using max rank r_max for each module
      - effective_params: using effective rank r_eff based on nonzero E
    Returns (full_params, effective_params).

    We deduplicate shared A/B/E by keying on (id(A), id(B), id(E)).
    """
    full_params = 0
    effective_params = 0
    seen = set()

    for _, module in model.named_modules():
        if not is_adalora_linear(module):
            continue

        # available adapters on this module
        adapters = list(getattr(module, "lora_A", {}).keys())
        if len(adapters) == 0:
            continue

        name = adapter_name if adapter_name in adapters else adapters[0]

        A = module.lora_A[name]
        B = module.lora_B[name]
        if hasattr(module, "lora_E") and name in module.lora_E:
            E = module.lora_E[name].view(-1)
        else:
            # if no E, treat all directions as active
            E = torch.ones(A.size(0), device=A.device)

        key = (id(A), id(B), id(E))
        if key in seen:
            continue
        seen.add(key)

        r_max = A.size(0)
        in_dim = A.size(1)
        out_dim = B.size(0)

        # full budget = max rank
        full_params += r_max * (in_dim + out_dim + 1)

        # effective rank: how many singular values are really used
        r_eff = int((E.abs() > thresh).sum().item())
        effective_params += r_eff * (in_dim + out_dim + 1)

    return full_params, effective_params
    
def train_eval_stage2(model, epochs=EPOCHS_STAGE2, batch_size=BATCH_SIZE_STAGE2, lr=1.2e-3,adalora_stage2=False,alpha=0):
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator,
    )
    valid_loader = DataLoader(
        valid_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=data_collator,
    )
    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=data_collator,
    )

    unique_params = {}
    for n, p in model.named_parameters():
        if p.requires_grad:
            unique_params[id(p)] = p
    optim = torch.optim.AdamW(list(unique_params.values()), lr=lr, weight_decay=0.01)

    total_steps = epochs * math.ceil(len(train_loader))
    sched = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps,
    )

    best_acc, best_state = 0.0, None
    global_step = 0

    def eval_loader(model, loader):
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                logits = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                ).logits
                pred = logits.argmax(-1)
                correct += (pred == batch["labels"]).sum().item()
                total += pred.numel()
        return correct / total

    model.train()
    for ep in range(epochs):
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**batch)
            loss = out.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step()

            # AdaLoRA rank allocation in Stage-2
            global_step += 1
            model.base_model.update_and_allocate(global_step)

            sched.step()
            optim.zero_grad()

        val_acc = eval_loader(model, valid_loader)
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
        print(f"Epoch {ep+1:02d} | val_acc={val_acc:.4f} | best={best_acc:.4f}")
        model.train()

    model.load_state_dict(best_state)
    model.to(device)
    if alpha=90:
        torch.save(best_state, "best_adalora_drop90.pt")
    elif alpha=95:
        torch.save(best_state, "best_adalora_drop90.pt")
    else:
        torch.save(best_state, "best_adalora_full.pt")
    if adalora_stage2:
        print(f"[Stage-2] Best dev accuracy (from best_state): {best_acc:.4f}")
    else:
        print(f"Best dev accuracy (from best_state): {best_acc:.4f}")

    final_val_acc = eval_loader(model, valid_loader)
    final_test_acc = eval_loader(model, test_loader)
    if adalora_stage2:
        print(f"[Stage-2] Final dev accuracy (recomputed): {final_val_acc:.4f}")
        print(f"[Stage-2] Test accuracy (best_state): {final_test_acc:.4f}")
    else:
        print(f"Final dev accuracy (recomputed): {final_val_acc:.4f}")
        print(f"Test accuracy (best_state): {final_test_acc:.4f}")

    full_budget, eff_budget = compute_adalora_budget(model, adapter_name="default", thresh=1e-6)
    if adalora_stage2:
        print(f"[Stage-2] AdaLoRA-drop full parameter budget (max rank): {full_budget}")
        print(f"[Stage-2] AdaLoRA-drop effective parameter budget (after pruning): {eff_budget}")
    else:
        print(f"AdaLoRA full parameter budget (max rank): {full_budget}")
        print(f"AdaLoRA effective parameter budget (after pruning): {eff_budget}")
    if full_budget > 0:
        if adalora_stage2:
            print(f"[Stage-2] AdaLoRA-drop compression ratio: {eff_budget / full_budget:.4f}")
        else:
            print(f"AdaLoRA compression ratio: {eff_budget / full_budget:.4f}")

    return final_val_acc, final_test_acc, eff_budget

# AdaLoRA-drop @ 95% 
stage2_model_drop95 = build_stage2_model()
shared_bank_95 = SharedAdaLoraBank(stage2_model_drop95).to(device)
stage2_model_drop95.shared_bank = shared_bank_95
apply_sharing_with_bank(stage2_model_drop95, low95_ids, shared_bank_95)

adalora95_val, adalora95_test, adalora95_eff = train_eval_stage2(
    stage2_model_drop95, epochs=5, batch_size=64, lr=1.2e-3,adalora_stage2=True, alpha= 95
)
print("[AdaLoRA-drop@95] dev_acc:", adalora95_val, "test_acc:", adalora95_test)

# AdaLoRA-drop @ 90% 
stage2_model_drop90 = build_stage2_model()
shared_bank_90 = SharedAdaLoraBank(stage2_model_drop90).to(device)
stage2_model_drop90.shared_bank = shared_bank_90
apply_sharing_with_bank(stage2_model_drop90, low90_ids, shared_bank_90)

adalora90_val, adalora90_test, adalora90_eff = train_eval_stage2(
    stage2_model_drop90, epochs=5, batch_size=64, lr=1.2e-3,adalora_stage2=True,alpha = 90
)
print("[AdaLoRA-drop@90] dev_acc:", adalora90_val, "test_acc:", adalora90_test)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Stage-2] Shared parameters tied across 43 low-importance modules.
Epoch 01 | val_acc=0.5640 | best=0.5640
Epoch 02 | val_acc=0.8640 | best=0.8640
Epoch 03 | val_acc=0.8780 | best=0.8780
Epoch 04 | val_acc=0.8700 | best=0.8780
Epoch 05 | val_acc=0.8820 | best=0.8820
[Stage-2] Best dev accuracy (from best_state): 0.8820


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Stage-2] Final dev accuracy (recomputed): 0.8820
[Stage-2] Test accuracy (best_state): 0.8744
[Stage-2] AdaLoRA-drop full parameter budget (max rank): 433380
[Stage-2] AdaLoRA-drop effective parameter budget (after pruning): 133698
[Stage-2] AdaLoRA-drop compression ratio: 0.3085
[AdaLoRA-drop@95] dev_acc: 0.882 test_acc: 0.874365739801096
[Stage-2] Shared parameters tied across 49 low-importance modules.
Epoch 01 | val_acc=0.6540 | best=0.6540
Epoch 02 | val_acc=0.8020 | best=0.8020
Epoch 03 | val_acc=0.8760 | best=0.8760
Epoch 04 | val_acc=0.8680 | best=0.8760
Epoch 05 | val_acc=0.8760 | best=0.8760
[Stage-2] Best dev accuracy (from best_state): 0.8760
[Stage-2] Final dev accuracy (recomputed): 0.8760
[Stage-2] Test accuracy (best_state): 0.8736
[Stage-2] AdaLoRA-drop full parameter budget (max rank): 336558
[Stage-2] AdaLoRA-drop effective parameter budget (after pruning): 66846
[Stage-2] AdaLoRA-drop compression ratio: 0.1986
[AdaLoRA-drop@90] dev_acc: 0.876 test_acc: 0.8735538867

In [13]:
##############################################
# BASELINE: Full RoBERTa-base
##############################################

from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
import torch, math

# assumes:
#   - model_name = "roberta-base"
#   - device already set
#   - train_ds, valid_ds, test_ds already built
#   - data_collator already defined (DataCollatorWithPadding)


def build_roberta_baseline():
    """
    Plain RoBERTa-base + classification head.
    All parameters are trainable (full fine-tuning).
    """
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=3,
    )
    model.to(device)
    return model


def train_eval_roberta_baseline(model, epochs=5, batch_size=64, lr=2e-5):
    """
    Train full RoBERTa-base on SciTail and return (val_acc, test_acc, trainable_params).
    - epochs, batch_size can be matched to your AdaLoRA run.
    - lr=2e-5 is a typical full FT LR; if you REALLY want
      'same everything' as AdaLoRA, set lr=1.2e-3 (but that may be unstable).
    """
    # DataLoaders (use same collator as your AdaLoRA code)
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator,
    )
    valid_loader = DataLoader(
        valid_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=data_collator,
    )
    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=data_collator,
    )

    # Optimizer on ALL model params
    optim = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=0.01,
    )

    # Scheduler (same style as your AdaLoRA code)
    total_steps = epochs * math.ceil(len(train_loader))
    sched = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps,
    )

    # Parameter counts for comparison
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"[Baseline] Total params:     {total_params}")
    print(f"[Baseline] Trainable params: {trainable_params}")

    def eval_loader(m, loader):
        m.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                # only pass what we need for logits
                logits = m(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                ).logits
                pred = logits.argmax(dim=-1)
                correct += (pred == batch["labels"]).sum().item()
                total += pred.numel()
        return correct / total

    best_acc, best_state = 0.0, None

    # -------- TRAINING LOOP --------
    for ep in range(epochs):
        model.train()
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            # model will use "labels" to compute loss
            out = model(**batch)
            loss = out.loss
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step()
            sched.step()
            optim.zero_grad()

        # validation accuracy
        val_acc = eval_loader(model, valid_loader)
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}

        print(f"[Baseline][Epoch {ep+1:02d}] val_acc={val_acc:.4f} | best={best_acc:.4f}")

    # restore best model
    if best_state is not None:
        model.load_state_dict(best_state)
        model.to(device)
        torch.save(best_state, "best_baseline.pt")

    # final dev & test accuracy
    final_val_acc = eval_loader(model, valid_loader)
    final_test_acc = eval_loader(model, test_loader)
    print(f"[Baseline] Final dev accuracy:  {final_val_acc:.4f}")
    print(f"[Baseline] Test accuracy:       {final_test_acc:.4f}")

    return final_val_acc, final_test_acc, trainable_params


##############################################
# Run baseline and compare to your AdaLoRA
##############################################

baseline_model = build_roberta_baseline()

# choose lr:
# - 2e-5 is safer for full FT
# - 1.2e-3 if you insist on exactly same LR as your AdaLoRA code
baseline_val_acc, baseline_test_acc, baseline_params = train_eval_roberta_baseline(
    baseline_model,
    epochs=5,
    batch_size=64,
    lr=2e-5,
)

print("\n===== COMPARISON =====")
print(f"Baseline dev acc     : {baseline_val_acc:.4f}")
print(f"Baseline test acc    : {baseline_test_acc:.4f}")
# assuming you already have these from your AdaLoRA run:
#   best_val_acc, test_acc, adalora_eff_params
# print(f"AdaLoRA dev acc      : {best_val_acc:.4f}")
# print(f"AdaLoRA test acc     : {test_acc:.4f}")
# print(f"Baseline params      : {baseline_params}")
# print(f"AdaLoRA eff. params  : {adalora_eff_params}")
# print(f"Compression ratio    : {adalora_eff_params / baseline_params:.4f}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Baseline] Total params:     124647939
[Baseline] Trainable params: 124647939
[Baseline][Epoch 01] val_acc=0.8560 | best=0.8560
[Baseline][Epoch 02] val_acc=0.8900 | best=0.8900
[Baseline][Epoch 03] val_acc=0.8800 | best=0.8900
[Baseline][Epoch 04] val_acc=0.8920 | best=0.8920
[Baseline][Epoch 05] val_acc=0.8920 | best=0.8920
[Baseline] Final dev accuracy:  0.8920
[Baseline] Test accuracy:       0.8863

===== COMPARISON =====
Baseline dev acc     : 0.8920
Baseline test acc    : 0.8863


In [14]:
# AdaLoRA-everywhere: no sharing, no dropping, rank allocated independently
stage2_model_adalora_full = build_stage2_model()

adalora_full_val, adalora_full_test, adalora_full_eff = train_eval_stage2(
    stage2_model_adalora_full, epochs=5, batch_size=64, lr=1.2e-3, alpha = 0
)
print("[AdaLoRA-all] dev_acc:", adalora_full_val, "test_acc:", adalora_full_test)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 01 | val_acc=0.8080 | best=0.8080
Epoch 02 | val_acc=0.8780 | best=0.8780
Epoch 03 | val_acc=0.8960 | best=0.8960
Epoch 04 | val_acc=0.8960 | best=0.8960
Epoch 05 | val_acc=0.8900 | best=0.8960
Best dev accuracy (from best_state): 0.8960
Final dev accuracy (recomputed): 0.8960
Test accuracy (best_state): 0.8805
AdaLoRA full parameter budget (max rank): 995760
AdaLoRA effective parameter budget (after pruning): 682272
AdaLoRA compression ratio: 0.6852
[AdaLoRA-all] dev_acc: 0.896 test_acc: 0.8804546377105744


In [15]:
from peft import LoraConfig
# =========================
# LoRA CONFIG (used for both LoRA-drop and LoRA-all)
# =========================
LORA_CFG = dict(
    r=4,
    lora_alpha=16,
    lora_dropout=0.0,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    target_modules=["query", "key", "value", "dense"],
)

def build_lora_stage1_model():
    base = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=3
    ).to(device)
    lora_cfg = LoraConfig(**LORA_CFG)
    model = get_peft_model(base, lora_cfg)
    model.print_trainable_parameters()
    return model

def build_lora_stage2_model():
    base = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=3
    ).to(device)
    lora_cfg = LoraConfig(**LORA_CFG)
    model = get_peft_model(base, lora_cfg)
    return model


In [16]:
def train_for_importance_lora(
    peft_model,
    alpha=ALPHA_STAGE1,
    epochs=EPOCHS_STAGE1,
    lr=1.2e-3,
    batch_size=BATCH_SIZE_STAGE1,
):
    n = len(train_ds)
    k = max(1, int(alpha * n))
    idx = list(range(n))
    random.shuffle(idx); idx = idx[:k]
    small = train_ds.select(idx)

    dl = DataLoader(
        small,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator,
        drop_last=False,
    )

    unique_params = {}
    for n, p in peft_model.named_parameters():
        if p.requires_grad:
            unique_params[id(p)] = p
    optim = torch.optim.AdamW(list(unique_params.values()), lr=lr)

    total_steps = epochs * math.ceil(len(dl))
    sched = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps,
    )

    energy, hooks = make_energy_meter(peft_model)
    peft_model.train()

    for ep in range(epochs):
        for batch in dl:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = peft_model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["labels"],
            )
            loss = out.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(peft_model.parameters(), 1.0)
            optim.step()
            sched.step()
            optim.zero_grad()

    remove_hooks(hooks)

    total_e = sum(energy.values()) + 1e-12
    importance = []
    for (full_name, group, tag), val in energy.items():
        importance.append({
            "name": full_name, "group": group, "tag": tag,
            "energy": val, "importance": val / total_e,
        })
    importance.sort(key=lambda d: d["importance"], reverse=True)
    return importance


In [17]:
class SharedLoraBank(nn.Module):
    """
    LoRA version: share A/B across low-importance modules in each group.
    Works whether lora_A / lora_B are nn.Linear modules or raw tensors.
    """
    def __init__(self, model, adapter_name="default"):
        super().__init__()
        self.adapter_name = adapter_name

        # pick one exemplar per group to get shapes
        exemplars = {}
        for name, m in model.named_modules():
            group, tag = classify_module_name(name)
            if group and group not in exemplars and is_adalora_linear(m) and len(m.lora_A) > 0:
                exemplars[group] = m

        for group, m in exemplars.items():
            adapter = self.adapter_name
            if adapter not in m.lora_A:
                adapter = next(iter(m.lora_A.keys()))

            A_obj = m.lora_A[adapter]
            B_obj = m.lora_B[adapter]

            # unwrap weights if they are Linear modules
            A_w = A_obj.weight if isinstance(A_obj, nn.Linear) else A_obj
            B_w = B_obj.weight if isinstance(B_obj, nn.Linear) else B_obj

            r, in_dim = A_w.shape
            out_dim, r2 = B_w.shape
            assert r == r2

            # shared parameters
            A_param = nn.Parameter(torch.empty_like(A_w))
            B_param = nn.Parameter(torch.empty_like(B_w))
            nn.init.kaiming_uniform_(A_param, a=math.sqrt(5))
            nn.init.zeros_(B_param)

            self.register_parameter(f"{group}_A", A_param)
            self.register_parameter(f"{group}_B", B_param)

    def tie_into(self, module, group):
        """
        Tie this module's LoRA A/B weights to the shared bank.
        """
        adapter = self.adapter_name
        if adapter not in module.lora_A:
            adapter = next(iter(module.lora_A.keys()))

        shared_A = getattr(self, f"{group}_A")
        shared_B = getattr(self, f"{group}_B")

        A_obj = module.lora_A[adapter]
        B_obj = module.lora_B[adapter]

        # If A/B are Linear modules, assign to their .weight;
        # otherwise, replace the stored tensor directly.
        if isinstance(A_obj, nn.Linear):
            A_obj.weight = shared_A
        else:
            module.lora_A[adapter] = shared_A

        if isinstance(B_obj, nn.Linear):
            B_obj.weight = shared_B
        else:
            module.lora_B[adapter] = shared_B


In [18]:
def train_eval_stage2_lora(model, epochs=EPOCHS_STAGE2, batch_size=BATCH_SIZE_STAGE2, lr=1.2e-3,lora_stage2=False):
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator,
    )
    valid_loader = DataLoader(
        valid_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=data_collator,
    )
    test_loader = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=data_collator,
    )

    unique_params = {}
    for n, p in model.named_parameters():
        if p.requires_grad:
            unique_params[id(p)] = p
    optim = torch.optim.AdamW(list(unique_params.values()), lr=lr, weight_decay=0.01)

    trainable_params = sum(p.numel() for p in unique_params.values())
    if lora_stage2:
        print(f"[LoRA-drop-Stage2] Trainable params (dedup): {trainable_params}")
    else:
        print(f"[LoRA] Trainable params (dedup): {trainable_params}")

    total_steps = epochs * math.ceil(len(train_loader))
    sched = get_linear_schedule_with_warmup(
        optim,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps,
    )

    best_acc, best_state = 0.0, None

    def eval_loader(model, loader):
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                logits = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                ).logits
                pred = logits.argmax(-1)
                correct += (pred == batch["labels"]).sum().item()
                total += pred.numel()
        return correct / total

    model.train()
    for ep in range(epochs):
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**batch)
            loss = out.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step()
            sched.step()
            optim.zero_grad()

        val_acc = eval_loader(model, valid_loader)
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
        if lora_stage2:
            print(f"[LoRA-drop-Stage2][Epoch {ep+1:02d}] val_acc={val_acc:.4f} | best={best_acc:.4f}")
        else:
            print(f"[LoRA][Epoch {ep+1:02d}] val_acc={val_acc:.4f} | best={best_acc:.4f}")

        model.train()

    model.load_state_dict(best_state)
    model.to(device)
    if lora_stage2:
        torch.save(best_state, "best_loradrop.pt")
    else:
        torch.save(best_state, "best_lora.pt")
    final_val_acc = eval_loader(model, valid_loader)
    final_test_acc = eval_loader(model, test_loader)
    if lora_stage2:
        print(f"[LoRA-drop-Stage2] Final dev accuracy:  {final_val_acc:.4f}")
        print(f"[LoRA-drop-Stage2] Test accuracy:       {final_test_acc:.4f}")
    else:
        print(f"[LoRA] Final dev accuracy:  {final_val_acc:.4f}")
        print(f"[LoRA] Test accuracy:       {final_test_acc:.4f}")

    return final_val_acc, final_test_acc, trainable_params


In [20]:
# ----- LoRA-drop @ 95% -----
lora_stage1_model = build_lora_stage1_model()
importance_lora = train_for_importance_lora(
    lora_stage1_model, alpha=0.10, epochs=3, lr=1.2e-3, batch_size=64
)
important_lora95_ids, low_lora95_ids, cum_lora95 = split_importance(importance_lora, 0.95)
print(f"[LoRA-Stage1@0.95] selected {len(important_lora95_ids)} important, {len(low_lora95_ids)} low; cum={cum_lora95:.3f}")

stage2_lora_drop95 = build_lora_stage2_model()
shared_lora_bank95 = SharedLoraBank(stage2_lora_drop95).to(device)
stage2_lora_drop95.shared_bank = shared_lora_bank95
apply_sharing_with_bank(stage2_lora_drop95, low_lora95_ids, shared_lora_bank95)

lora_drop95_val, lora_drop95_test, lora_drop95_params = train_eval_stage2_lora(
    stage2_lora_drop95, epochs=5, batch_size=64, lr=1.2e-3, lora_stage2 = True
)
print("[LoRA-drop@95] dev_acc:", lora_drop95_val, "test_acc:", lora_drop95_test)

# ----- LoRA-everywhere -----
stage2_lora_full = build_lora_stage2_model()
lora_full_val, lora_full_test, lora_full_params = train_eval_stage2_lora(
    stage2_lora_full, epochs=5, batch_size=64, lr=1.2e-3
)
print("[LoRA-all] dev_acc:", lora_full_val, "test_acc:", lora_full_test)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,256,451 || all params: 125,904,390 || trainable%: 0.9979


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[LoRA-Stage1@0.95] selected 39 important, 33 low; cum=0.952
[Stage-2] Shared parameters tied across 33 low-importance modules.
[LoRA-drop-Stage2] Trainable params (dedup): 1062915
[LoRA-drop-Stage2][Epoch 01] val_acc=0.8820 | best=0.8820
[LoRA-drop-Stage2][Epoch 02] val_acc=0.8880 | best=0.8880
[LoRA-drop-Stage2][Epoch 03] val_acc=0.9040 | best=0.9040
[LoRA-drop-Stage2][Epoch 04] val_acc=0.8920 | best=0.9040
[LoRA-drop-Stage2][Epoch 05] val_acc=0.9060 | best=0.9060


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[LoRA-drop-Stage2] Final dev accuracy:  0.9060
[LoRA-drop-Stage2] Test accuracy:       0.8900
[LoRA-drop@95] dev_acc: 0.906 test_acc: 0.8899939111020905
[LoRA] Trainable params (dedup): 1256451
[LoRA][Epoch 01] val_acc=0.8560 | best=0.8560
[LoRA][Epoch 02] val_acc=0.8920 | best=0.8920
[LoRA][Epoch 03] val_acc=0.8840 | best=0.8920
[LoRA][Epoch 04] val_acc=0.9000 | best=0.9000
[LoRA][Epoch 05] val_acc=0.9100 | best=0.9100
[LoRA] Final dev accuracy:  0.9100
[LoRA] Test accuracy:       0.8939
[LoRA-all] dev_acc: 0.91 test_acc: 0.8938502131114269


In [25]:
import pandas as pd

# Create DataFrame with your variables
df = pd.DataFrame({
    'Model': [
        'AdaLoRA-drop@95',
        'AdaLoRA-drop@90', 
        'Baseline',
        'AdaLoRA-all',
        'LoRA-drop@95',
        'LoRA-all'
    ],
    'Parameters': [
        133698,
        66846,
        124647939,
        682272,  
        1062915,
        1256451
    ],
    'Test Accuracy': [
        adalora95_test,
        adalora90_test,
        baseline_test_acc,
        adalora_full_test,
        lora_drop95_test,
        lora_full_test
    ]
})

# Format for clean display
df['Parameters'] = df['Parameters'].apply(
    lambda x: f"{x:,}" if pd.notna(x) else "N/A"
)
df['Test Accuracy'] = df['Test Accuracy'].apply(lambda x: f"{x:.4f}")

# Display without index
print(df.to_string(index=False))

          Model  Parameters Test Accuracy
AdaLoRA-drop@95     133,698        0.8744
AdaLoRA-drop@90      66,846        0.8736
       Baseline 124,647,939        0.8863
    AdaLoRA-all     682,272        0.8805
   LoRA-drop@95   1,062,915        0.8900
       LoRA-all   1,256,451        0.8939
